Visualización de Datos en Tiempo Real con Grafana
Con los datos fluyendo y guardándose en la base de datos, el siguiente paso es darles vida mediante gráficos. Esto nos permitirá monitorear las cámaras de frío en tiempo real.

1. Acceso a Grafana
Entraremos a la interfaz de Grafana a través del navegador utilizando la IP de la Raspberry Pi y el puerto 3000:

Acceso: http://192.168.100.53:3000

Credenciales iniciales: El usuario por defecto es admin y la contraseña es admin. Al ingresar por primera vez, el sistema te pedirá obligatoriamente cambiarla por una más segura.

2. Conexión con la Base de Datos (Data Source)
Una vez dentro, debemos "conectar" Grafana con nuestra base de datos PostgreSQL:

Ve al panel de la izquierda, selecciona Connections (Conexiones) y luego Data Sources.

Haz clic en Add data source y busca PostgreSQL.

Configura los siguientes campos con los datos que definimos en nuestro archivo de Docker:

Host URL: db:5432 (Usamos "db" porque es el nombre del servicio en nuestro Docker Compose).

Database: iot_db.

User / Password: Las credenciales que configuraste para tu base de datos.

SSL Mode: Cambia este ajuste a Disable.

Nota técnica: Desactivamos el SSL porque toda la comunicación ocurre de manera interna dentro de la Raspberry Pi (red local). Esto reduce la carga de procesamiento en la Raspberry, optimizando su rendimiento sin comprometer la seguridad, ya que los datos no salen a internet.

Al finalizar, presiona el botón Save & Test. Deberia aparecer un mensaje en verde.

Creación del Dashboard y Visualización
Con la conexión establecida, ya podemos transformar los datos brutos en información visual. Para ello, seguiremos estos pasos:

Crear el Dashboard: En el panel izquierdo, seleccionamos la opción Dashboards y luego hacemos clic en New > Add visualization.

Seleccionar la Fuente de Datos: En la lista desplegable, elegimos la conexión de PostgreSQL que configuramos en el paso anterior.

Configurar la Visualización: Al principio aparecerá un gráfico vacío. En la parte inferior encontraremos el editor de consultas. Aunque Grafana permite seleccionar columnas de forma manual (modo visual), yo utilizé el modo SQL Query.

Para graficar la temperatura en el tiempo, activamos el modo Code e ingresamos la siguiente consulta:

In [ ]:
SELECT
  fecha_registro AS "time",
  temp_interior AS "Interior",
  temp_exterior AS "Exterior"
FROM lecturas_sensor
WHERE $__timeFilter(fecha_registro)
ORDER BY 1

Con la consulta funcionando, el gráfico ya es visible, pero podemos llevarlo a un nivel más profesional utilizando las herramientas de diseño del panel derecho. En mi configuración, realicé los siguientes ajustes para facilitar la toma de decisiones:

1. Umbrales y Líneas de Referencia (Thresholds)
Para una cámara de frío, los límites visuales son críticos. Añadí líneas horizontales estratégicas:

Línea de 0°C (Triple): Coloqué una marca clara para monitorear cuántas veces la temperatura cruza el punto de congelación.

Línea de 5°C (Alerta): Sirve como una frontera de seguridad; si la temperatura toca esta línea, sé que debo actuar.

Línea de 30°C (Calor): solo lo puse para tener un indicio de cuanto calor hace afuera de la cámara

2. Estilo Visual
Para mejorar la legibilidad, activé el relleno de área (Opacity) debajo de las líneas y suavicé las curvas, lo que permite identificar tendencias de un solo vistazo sin saturar la vista.

3. Estadísticas de Consulta Rápida (Legend Mode)
En la parte inferior del gráfico, configuré la leyenda en modo tabla para mostrar:

Promedio (Avg): Para entender el comportamiento general del clima en el periodo seleccionado.

Último dato (Last): Es el valor más importante; la temperatura actual exacta de la cámara.

Estas métricas son fundamentales para una visualización rápida, permitiendo que cualquier persona que mire la pantalla entienda el estado del sistema en menos de 5 segundos.